In [1]:
import os
import time
import torch
import torchvision.transforms as transforms
import numpy as np
import glob
from PIL import Image

In [2]:
from google.colab import drive
drive.mount("/content/drive")
import sys

filepath = "/content/drive/MyDrive/code_and_data"  # Change this path for your Google drive setting.
sys.path.append(filepath)

from model_baseline import cancer_classifier
# Upload util.py and model_baseline.py on your Google Drive filepath.

Mounted at /content/drive


In [3]:
class cancer_test_dataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.files = glob.glob(f"{root}/*.jpg")
    
    def __len__(self):
        ans = len(self.files)
        return ans

    def __getitem__(self, idx):
        image = Image.open(f"{self.root}/{str(idx + 1)}.jpg")
        if self.transform is not None:
            image = self.transform(image)
        return image

In [4]:
state_path = f"{filepath}/third_run/model_state_dict.pt"  # Write down your model location
experiment = "third_run"  # Write down experiment name (For saving the result)

model = cancer_classifier.cancer_classifier_function()
print(model)
model.load_state_dict(torch.load(state_path))
model.eval()

# If loading is done with no error, you will see  " <All keys matched successfully> "

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
## Data load
test_dataset = cancer_test_dataset(root=f"{filepath}/test/",
                                                transform=transforms.Compose([
                                                    transforms.ToTensor()
                                                ]))
test_loader = torch.utils.data.DataLoader(test_dataset,
                                        batch_size=4,
                                        shuffle=False,
                                        drop_last=False
                                        )

In [6]:
prediction_index = []
accuracy_list = []

for _, (batch) in enumerate(test_loader):
    batch = batch.cuda()
    prediction = model(batch)
    _, i = prediction.max(dim=1)
    prediction_index += i.tolist()

final_index = prediction_index
print(f"Example prediction, 5 items from front : {final_index[:5]}")
print(f"""{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())} || Test finished!""")
np.savetxt(f"{filepath}/{experiment}/test_result.csv", np.dstack((np.arange(1, len(final_index)+1), final_index))[0], "%d,%d", header="Id,Values")

Example prediction, 5 items from front : [0, 0, 1, 1, 1]
2020-12-13 13:24:21 || Test finished!
